## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2025">The Repo</a>.  

Once you have updated your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://github.com/apps/databricks">Databricks App on Github</a> or by cloning the repo to your laptop and then uploading the final_project directory and its contents to your workspace using file imports.  Your choice.

<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches to triggering your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)
- [In class examples - Spark Structured Streaming Performance](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/2638424645880316?o=1093580174577663)

### Be sure your project runs end to end when *Run all* is executued on this notebook! (7 points)

### This project is worth 25% of your final grade.
- DSCC-202 Students have 55 possible points on this project (see points above and the instructions below)
- DSCC-402 Students have 60 possible points on this project (one extra section to complete)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    # Optimize the tables
    optimize_table(BRONZE_DELTA)
    optimize_table(SILVER_DELTA)
    optimize_table(GOLD_DELTA)
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here (2 points)
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

# Delta Lake
from delta.tables import DeltaTable

# MLflow
import mlflow
import mlflow.spark
import mlflow.pyfunc

# Data science utilities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# Python stdlib
import time
from datetime import datetime


spark.conf.set("spark.sql.shuffle.partitions", "16")    
spark.conf.set("spark.default.parallelism", "16")
spark.conf.set("spark.sql.adaptive.enabled", "true")

## 2.0 Define and execute utility functions (3 points)
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
def inspect_raw_tweets(path: str, sample_index: int = 0, head_chars: int = 1000):
    """
    Lists and counts files under `path`, displays the file list,
    and prints the first `head_chars` characters of the file at
    position `sample_index`.
    """
    files = dbutils.fs.ls(path)
    count = len(files)
    print(f"Total source files found: {count}")

    if count == 0:
        print("No files found in the source directory.")
        return files

    if files:
        idx = sample_index % count
        sample_path = files[idx].path
        print(f"\n--- Sample file [{idx}] path: {sample_path} ---\n")
        print(dbutils.fs.head(sample_path, head_chars))

    return files

In [0]:
source_files = inspect_raw_tweets(TWEET_SOURCE_PATH, sample_index=0, head_chars=1000)

## 3.0 Transform the Raw Data to Bronze Data using a stream  (8 points)
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using delta lake to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defined in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Schema for the raw tweet JSON
raw_schema = T.StructType([
    T.StructField("date", T.StringType(), True),
    T.StructField("user", T.StringType(), True),
    T.StructField("text", T.StringType(), True),
    T.StructField("sentiment", T.StringType(), True)
])
# Read stream from source using cloudfiles
raw_df = (
    spark.readStream
         .format("cloudFiles")
         .option("cloudFiles.format", "json")
         .option("cloudFiles.schemaLocation", USER_DIR + "bronze_schema")
         .option("maxFilesPerTrigger", 100)
         .schema(raw_schema)
         .load(TWEET_SOURCE_PATH)
)
# Add metadata
bronze_df = raw_df.select(
    F.col("date"),
    F.col("user"),
    F.col("text"),
    F.col("sentiment")
).withColumn("source_file", F.input_file_name()) \
 .withColumn("processing_time", F.current_timestamp())

# Assign bronze tag 
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "bronze")

#  Write the stream out to a Delta table (append mode)
bronze_stream = (
    bronze_df.writeStream
             .format("delta")
             .option("checkpointLocation", BRONZE_CHECKPOINT)
             .option("mergeSchema", "true")
             .trigger(processingTime="30 seconds")
             .outputMode("append")
             .queryName("bronze_stream")
             .start(BRONZE_DELTA)
)

print(bronze_stream)


In [0]:
if wait_for_delta_table(BRONZE_DELTA, timeout=120, check_interval=3):
    print("✅ Bronze table ready.")
else:
    raise RuntimeError("🛑 Bronze table never materialized!")

## 4.0 Transform the Bronze Data to Silver Data using a stream (5 points)
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Read a streaming DataFrame off the bronze Delta table
bronze_stream_df = (
    spark.readStream
         .format("delta")
         .load(BRONZE_DELTA)
)

# Preprocess:
#    - parse the date string into a proper timestamp
#    - remove all "@username" mentions from the text to create cleaned_text
#    - extract an array of tokens that start with "@" as mentions

silver_base = (
    bronze_stream_df
      .withColumn("timestamp", F.to_timestamp(F.col("date")))
      .withColumn("cleaned_text", F.regexp_replace(F.col("text"), "@\\w+", ""))
      .withColumn("mentions", F.expr("filter(split(text, ' '), x -> x like '@%')"))
)

# Explode the mentions array so that each mention yields its own row

silver_df = (silver_base.select(
    F.col("timestamp"),
    F.col("sentiment"),
    F.explode(F.col("mentions")).alias("mention"),
    F.col("cleaned_text")
    )
    .filter(F.col("mention").rlike(r"^@[A-Za-z0-9_]+$"))
)

# Assign silver tag 
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "silver")

# Write the stream to the silver Delta table
silver_stream = (
    silver_df.writeStream
            .format("delta")
            .option("checkpointLocation", SILVER_CHECKPOINT)
            .trigger(processingTime="20 seconds")
            .option("mergeSchema", "true")
            .outputMode("append")
            .queryName("silver_stream")
            .start(SILVER_DELTA)
)

print(silver_stream)

In [0]:
if wait_for_delta_table(SILVER_DELTA, timeout=60, check_interval=5):
    print("✅ Silver table ready.")
else:
    raise RuntimeError("🛑 Silver table never materialized!")

## 5.0 Transform the Silver Data to Gold Data using a stream (7 points)
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# Read a streaming DataFrame off the silver Delta table
silver_input = (
    spark.readStream
         .format("delta")
         .load(SILVER_DELTA)
)

# Load the MLflow model as a Spark UDF
MODEL_URI = f"models:/{MODEL_NAME}/Production"
sentiment_udf = mlflow.pyfunc.spark_udf(
    spark,
    model_uri=MODEL_URI,
    env_manager="local"
)

# Apply UDF and format data
gold_df = (
    silver_input
      # Call the UDF
      .withColumn("out", sentiment_udf(F.col("cleaned_text")))
      .withColumn("predicted_score", F.col("out.score") * 100)
      .withColumn("raw_pred_label", F.col("out.label"))
      # Normalize into three classes (positive, neutral, negative)
      .withColumn(
          "predicted_sentiment",
          F.when(F.col("out.label") == "NEG", "negative")
          .when(F.col("out.label") == "NEU", "neutral")
         .otherwise("positive")        
      )
      # map to IDs only positive and negative
      .withColumn(
         "predicted_sentiment_id",
         F.when(F.col("predicted_sentiment") == "negative", 0)
         .otherwise(1) # neutral & positive → 1
      )
      .withColumn(
         "sentiment_id",
         F.when(F.col("sentiment") == "negative", 0)
         .otherwise(1)
      )
      # select the final schema
      .select(
         "timestamp","mention","cleaned_text","sentiment",
         "predicted_score","predicted_sentiment","predicted_sentiment_id",
         "sentiment_id"
      )
)

# Assign gold tag 
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "gold")

# Write the stream to the gold Delta table
gold_stream = (
    gold_df.writeStream
           .format("delta")
           .option("checkpointLocation", GOLD_CHECKPOINT)
           .trigger(processingTime="60 seconds")
           .outputMode("append")
           .queryName("gold_stream")
           .start(GOLD_DELTA)
)

print(gold_stream)


In [0]:
if wait_for_delta_table(GOLD_DELTA, timeout=180, check_interval=5):
    print("✅ Gold table ready.")
else:
    raise RuntimeError("🛑 Gold table never materialized!")

## 6.0 Monitor your Streams (5 points)
- Setup a loop that runs at least every 10 seconds
- Print a timestamp of the monitoring query along with the list of streams, rows processed on each, and the processing time on each
- Run the loop until all of the data is processed (0 rows read on each active stream)
- Plot a line graph that shows the data processed by each stream over time
- Plot a line graph that shows the average processing time on each stream over time

In [0]:
monitoring_log = []

# 1) Run until all streams read 0 new rows
while True:
    stats_df = get_streaming_stats()   
    now = datetime.now()
    print(f"\nMonitoring at {now}")
    for _, row in stats_df.iterrows():
        print(f"{row['query']}: rows={row['input_rows']}, proc_time={row['processing_time']} ms")
    stats_df['monitoring_time'] = now
    monitoring_log.append(stats_df)
    if (stats_df.groupby('query')['input_rows'].last() == 0).all():
        break
    time.sleep(10)

# Combine all snapshots
log_df = pd.concat(monitoring_log)

# Aggregate to remove duplicate (monitoring_time, query) entries
agg_df = (
    log_df
      .groupby(['monitoring_time', 'query'], as_index=False)
      .agg({
          'input_rows': 'sum',          
          'processing_time': 'mean'
      })
)

# Plotting
pivot_rows = agg_df.pivot(index='monitoring_time', columns='query', values='input_rows')
pivot_proc = agg_df.pivot(index='monitoring_time', columns='query', values='processing_time')

ax1 = pivot_rows.plot(figsize=(10, 5))
ax1.set_title("Rows Processed Over Time")
ax1.set_xlabel("Time")
ax1.set_ylabel("Input Rows")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

ax2 = pivot_proc.plot(figsize=(10, 5))
ax2.set_title("Processing Time Over Time")
ax2.set_xlabel("Time")
ax2.set_ylabel("Processing Time (ms)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7.0 Bronze Data Exploratory Data Analysis (5 points)
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# Load the Bronze Delta table
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)

# Total number of tweets
total_tweets = bronze_df.count()
print(f"Total tweets in Bronze table: {total_tweets}")

# Count NULL values per column 
null_counts = bronze_df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in bronze_df.columns
])
print("Null counts per column:")
null_counts.show()

# Tweets per user, descending
user_tweet_counts = (
    bronze_df.groupBy("user")
             .count()
             .orderBy(F.col("count").desc())
)
print("Top 20 users by tweet count:")
display(user_tweet_counts.limit(20))

# Tweets with vs. without mentions
has_mention = F.when(F.col("text").contains("@"), 1).otherwise(0)
counts = (
    bronze_df
      .withColumn("has_mention", has_mention)
      .groupBy("has_mention")
      .count()
      .collect()
)
num_with  = next(r["count"] for r in counts if r["has_mention"] == 1)
num_without = next(r["count"] for r in counts if r["has_mention"] == 0)
print(f"Tweets with ≥1 mention: {num_with}")
print(f"Tweets with 0 mentions: {num_without}")

# Plot top 20 tweeters
top20 = user_tweet_counts.limit(20).toPandas()
plt.figure(figsize=(12, 6))
plt.bar(top20["user"], top20["count"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 20 Tweeters by Tweet Count")
plt.xlabel("User Handle")
plt.ylabel("Number of Tweets")
plt.tight_layout()
plt.show()


## 8.0 Capture the accuracy metrics from the gold table in MLflow  (4 points)
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the model name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# Load the Gold Delta table
gold_df = spark.read.format("delta").load(GOLD_DELTA)

# 2) Filter out any rows with NULL true or predicted labels
clean_gold = gold_df.filter(
   F.col("sentiment_id").isNotNull() & F.col("predicted_sentiment_id").isNotNull()
)

# 3) Select true and predicted labels
labels_df = clean_gold.select(
    F.col("sentiment_id").alias("true"),
    F.col("predicted_sentiment_id").alias("pred")
)

# 4) Convert to Pandas and extract NumPy arrays
pandas_df = labels_df.toPandas()
y_true = pandas_df["true"].astype(int).values
y_pred = pandas_df["pred"].astype(int).values

# 5) Compute precision, recall, and F1‑score (macro‑averaged)
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, average="macro", zero_division=0
)

# 6) Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# 7) Plot and save confusion matrix
fig, ax = plt.subplots(figsize=(4, 4))
im = ax.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
classes = ["negative", "positive"]
ax.set_xticks(np.arange(len(classes)))
ax.set_yticks(np.arange(len(classes)))
ax.set_xticklabels(classes)
ax.set_yticklabels(classes)
ax.set_xlabel("Predicted label")
ax.set_ylabel("True label")
ax.set_title("Confusion Matrix")
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j],
                ha="center", va="center",
                color="white" if cm[i, j] > cm.max() / 2 else "black")
plt.tight_layout()
conf_matrix_path = "/tmp/confusion_matrix.png"
fig.savefig(conf_matrix_path)
plt.close(fig)

# 8) Retrieve Delta table version for the silver input
silver_table = DeltaTable.forPath(spark, SILVER_DELTA)
silver_version = silver_table.history(1).select("version").first()["version"]

# 9) Log everything to MLflow
with mlflow.start_run() as run:
    # Parameters
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("mlflow_version", mlflow.__version__)
    mlflow.log_param("silver_delta_version", silver_version)
    # Metrics
    mlflow.log_metric("precision_macro", precision)
    mlflow.log_metric("recall_macro", recall)
    mlflow.log_metric("f1_macro", f1)
    # Artifact
    mlflow.log_artifact(conf_matrix_path, artifact_path="confusion_matrix")
    print(f"Logged MLflow run: {run.info.run_id}")

## 9.0 Application Data Processing and Visualization (6 points)
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# Load the Gold Delta table
gold_df = spark.read.format("delta").load(GOLD_DELTA)

# Total mentions (rows) in gold
total_mentions = gold_df.count()
print(f"Total mention‐rows in Gold data: {total_mentions}")

# Aggregate counts of each sentiment per mention
agg_df = (
    gold_df.groupBy("mention")
           .agg(
               F.sum(F.when(F.col("predicted_sentiment") == "negative", 1).otherwise(0)).alias("negative"),
               F.sum(F.when(F.col("predicted_sentiment") == "neutral",  1).otherwise(0)).alias("neutral"),
               F.sum(F.when(F.col("predicted_sentiment") == "positive", 1).otherwise(0)).alias("positive")
           )
           # 4) Total tweets per mention
           .withColumn("total", F.col("negative") + F.col("neutral") + F.col("positive"))
)

# Sort by total descending
sorted_df = agg_df.orderBy(F.col("total").desc())

# Show top 10 mentions by total
print("Top 10 mentions by total tweets:")
sorted_df.show(10, truncate=False)

# Plot top 20 mentions with highest positive counts
top20_pos = sorted_df.orderBy(F.col("positive").desc()).limit(20).toPandas()
plt.figure(figsize=(12, 6))
plt.bar(top20_pos["mention"], top20_pos["positive"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 20 Mentions by Positive Tweet Count")
plt.xlabel("Mention")
plt.ylabel("Positive Tweets")
plt.tight_layout()
plt.show()

# Plot top 20 mentions with highest negative counts
top20_neg = sorted_df.orderBy(F.col("negative").desc()).limit(20).toPandas()
plt.figure(figsize=(12, 6))
plt.bar(top20_neg["mention"], top20_neg["negative"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 20 Mentions by Negative Tweet Count")
plt.xlabel("Mention")
plt.ylabel("Negative Tweets")
plt.tight_layout()
plt.show()

## 10.0 Clean up and completion of your pipeline (3 points)
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook. Note: In the includes there is a variable START_TIME that captures the starting time of the notebook.

In [0]:
# List any active streams
active = [s.name for s in spark.streams.active]
print("Active streams before stopping:", active)

# Stop all active streams
if stop_all_streams():
    print("Stopped all active streams.")
else:
    print("No active streams to stop.")

# Calculate and print elapsed time
elapsed = time.time() - START_TIME
print(f"Total elapsed time: {elapsed:.2f} seconds")

## 11.0 How Optimized is your Spark Application (Grad Students Only) (5 points)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE